In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:85% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

In [20]:
# 생성형AI.ipynb 내용 (Jupyter용 파이썬 코드)
# 평가 요구사항에 맞는 LangChain 기반 음식 추천 + 레시피 생성 시스템

!pip install openai langchain python-dotenv

import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SequentialChain

# 1. 환경변수 로드 및 LLM 설정
load_dotenv()
llm = ChatOpenAI(temperature=0.7, model="gpt-4o")

# 2. food_chain: 나라명을 입력받아 대표 음식 추천
food_prompt = ChatPromptTemplate.from_template(
    """
    사용자가 입력한 나라: {country}
    해당 나라의 가장 유명한 음식 한 가지를 추천해주세요. 음식명만 출력하세요.
    """
)
food_chain = LLMChain(llm=llm, prompt=food_prompt, output_key="food")

# 3. recipe_chain: 음식명을 입력받아 레시피 출력
recipe_prompt = ChatPromptTemplate.from_template(
    """
    음식 이름: {food}
    위 음식을 집에서 만들 수 있는 레시피를 알려주세요. 재료와 순서를 포함하세요.
    """
)
recipe_chain = LLMChain(llm=llm, prompt=recipe_prompt, output_key="recipe")

# 4. 두 체인 연결: 나라 → 음식 → 레시피
full_chain = SequentialChain(
    chains=[food_chain, recipe_chain],
    input_variables=["country"],
    output_variables=["food", "recipe"],
    verbose=True
)

# 5. 실행 예시 (사용자 입력)
input_country = input("나라 이름을 입력하세요: ")
result = full_chain({"country": input_country})

# 6. 결과 출력
print(f"추천 음식: {result['food']}")
print("\n해당 음식의 레시피:")
print(result['recipe'])


나라 이름을 입력하세요: 베트남


> Entering new SequentialChain chain...

> Finished chain.
추천 음식: 포보.

해당 음식의 레시피:
포보(Phở bò)는 베트남의 전통적인 쇠고기 쌀국수 요리입니다. 집에서 포보를 만들기 위해 필요한 재료와 조리 순서를 아래에 소개합니다.

### 재료:

#### 육수:
- 쇠고기 뼈: 1.5kg
- 양파: 1개 (반으로 자름)
- 생강: 1큰술 (슬라이스)
- 팔각: 2~3개
- 정향: 4~5개
- 계피 스틱: 1개
- 설탕: 1큰술
- 피쉬 소스: 3~4큰술
- 소금: 적당량
- 물: 약 4~5리터

#### 국수와 고명:
- 쌀국수(포) 면: 500g
- 쇠고기 등심 또는 안심: 300g (얇게 슬라이스)
- 숙주나물: 200g
- 실파 또는 쪽파: 2~3대 (잘게 썰음)
- 고수: 적당량 (생략 가능)
- 라임: 1개 (조각으로 자름)
- 청양고추: 1~2개 (얇게 슬라이스, 선택 사항)

### 조리 순서:

1. **육수 준비:**
   - 큰 냄비에 쇠고기 뼈를 넣고 물을 부어 중불에서 끓입니다. 물이 끓어오르면 불순물을 제거하기 위해 뼈를 한번 데쳐줍니다.
   - 뼈를 깨끗이 씻고, 다시 4~5리터의 물을 부어 냄비에 넣습니다.
   - 양파, 생강을 직화 또는 팬에서 살짝 구워서 육수에 넣습니다.
   - 팔각, 정향, 계피도 함께 넣고, 설탕과 소금, 피쉬 소스를 추가하여 약 3~4시간 동안 중약불에서 끓입니다. 중간중간 거품과 기름을 제거해 줍니다.

2. **고기 준비:**
   - 쇠고기 등심 또는 안심을 최대한 얇게 슬라이스하여 준비합니다.

3. **면 삶기:**
   - 쌀국수 면을 포장지의 지시에 따라 삶은 후 찬물에 헹구어 물기를 뺍니다.

4. **국수 완성:**
   - 그릇에 삶은 쌀국수 면을 담고 얇게 슬라이스한 쇠고기를 얹습니다.
   - 끓여낸 육수를 체에 걸러 맑은 국물을 쇠고기와 면 위에 부어줍니다. 뜨거운 육수가 얇은 쇠고기를